In [1]:
import os

import MySQLdb as mdb
import pandas as pd
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, text

In [2]:
def create_connection():
    default_file = os.path.join(os.path.expanduser('~/'), '.atefd.cnf')
    return mdb.connect(read_default_file=default_file, db='EFD')

In [3]:
base = automap_base()
engine = create_engine('mysql+mysqldb://', creator=create_connection)
base.prepare(engine, reflect=True)
session = Session(engine)

In [4]:
cscs = ['atcamera', 'atHeaderService', 'atArchiver']
topic = 'logevent_summaryState'
df_columns = ['date_time', 'summaryState', 'priority']
cols1 = 'date_time,summaryStateValue,priority'
cols2 = ','.join(df_columns)
columns = [cols1, cols2, cols1]
limit = 'date_time desc limit 1'
tables = [base.classes.get('{}_{}'.format(csc, topic)) for csc in cscs]

In [5]:
df_rows = []
for table, cols, csc in zip(tables, columns, cscs):
    result = session.query(table).order_by(text(limit)).one()
    row = []
    for col in cols.split(','):
        row.append(getattr(result, col))
    df_rows.append(pd.DataFrame([row], columns=df_columns, index=[csc]))
df = pd.concat(df_rows)

In [6]:
df

,date_time,summaryState,priority
atcamera,2018-08-25 20:37:18.784911,2,1
atHeaderService,2018-08-26 23:33:27.293528,2,0
atArchiver,2018-08-26 23:27:16.853129,2,0
